### Build a Question Answering application over a Graph Database

In [1]:
NEO4J_URI= "neo4j+s://c83b5c27.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "DVZLAV-xSNgarAF6QPOhu3vDmrIyVoR9ussQlqZravM"
NEO4J_DATABASE= "neo4j"

In [2]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD
os.environ["NEO4J_DATABASE"] = NEO4J_DATABASE

In [3]:
from langchain_neo4j import Neo4jGraph
graph = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD)
graph

v:\Genai_course\Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
## Dataset Moview 
moview_query="""
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row

MERGE(m:Movie{id:row.movieId})
SET m.released = date(row.released),
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)
FOREACH (director in split(row.director, '|') | 
    MERGE (p:Person {name:trim(director)})
    MERGE (p)-[:DIRECTED]->(m))
FOREACH (actor in split(row.actors, '|') | 
    MERGE (p:Person {name:trim(actor)})
    MERGE (p)-[:ACTED_IN]->(m))
FOREACH (genre in split(row.genres, '|') | 
    MERGE (g:Genre {name:trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g))


"""

In [12]:
moview_query

"\nLOAD CSV WITH HEADERS FROM\n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/main/movies/movies_small.csv' as row\n\nMERGE(m:Movie{id:row.movieId})\nSET m.released = date(row.released),\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\nFOREACH (director in split(row.director, '|') | \n    MERGE (p:Person {name:trim(director)})\n    MERGE (p)-[:DIRECTED]->(m))\nFOREACH (actor in split(row.actors, '|') | \n    MERGE (p:Person {name:trim(actor)})\n    MERGE (p)-[:ACTED_IN]->(m))\nFOREACH (genre in split(row.genres, '|') | \n    MERGE (g:Genre {name:trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g))\n\n\n"

In [13]:
graph.query(moview_query)

[]

In [14]:
graph.refresh_schema()
print(graph.schema)

Node properties:
CEO {name: STRING, POB: STRING, YOB: INTEGER}
Company {name: STRING}
Movie {id: STRING, released: DATE, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [15]:
from dotenv import load_dotenv
load_dotenv()

True

In [25]:
from langchain_groq import ChatGroq

llm = ChatGroq(model="llama-3.1-8b-instant")

In [26]:
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023E32193B30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023E34090FE0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [27]:
from langchain_neo4j import GraphCypherQAChain

chain = GraphCypherQAChain.from_llm(llm=llm, graph=graph, verbose=True,allow_dangerous_requests=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_neo4j.graphs.neo4j_graph.Neo4jGraph object at 0x0000023E2B7B32F0>, cypher_generation_chain=PromptTemplate(input_variables=['question', 'schema'], input_types={}, partial_variables={}, template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}')
| RunnableBinding(bound=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023E32193B30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023E34090FE0>, model_name='llama-3.1-8b-in

In [28]:
response = chain.invoke({"query":"Who was the director of the movie casino?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:DIRECTED]->(m:Movie {title: "Casino"}) RETURN p.name


[#D4C7]  _: <CONNECTION> error: Failed to read from defunct connection ResolvedIPv4Address(('34.126.64.110', 7687)) (ResolvedIPv4Address(('34.126.64.110', 7687))): OSError('No data')
Unable to retrieve routing information
Transaction failed and will be retried in 1.0284286318027198s (Unable to retrieve routing information)
[#D490]  _: <CONNECTION> error: Failed to read from defunct connection IPv4Address(('si-c83b5c27-3c80.production-orch-0064.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.64.110', 7687))): OSError('No data')
Transaction failed and will be retried in 2.2670185297755516s (Failed to read from defunct connection IPv4Address(('si-c83b5c27-3c80.production-orch-0064.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.64.110', 7687))))


Full Context:
[{'p.name': 'Martin Scorsese'}]

> Finished chain.


{'query': 'Who was the director of the movie casino?',
 'result': 'Martin Scorsese was the director of the movie casino.'}

In [29]:
response=chain.invoke({"query":"Who were the actors of the movie Casino"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie)-[:ACTED_IN]->(p:Person), (m)<-[:DIRECTED]-(d:Person) WHERE m.title = "Casino" RETURN p.name;
Full Context:
[]

> Finished chain.


{'query': 'Who were the actors of the movie Casino',
 'result': "I don't know the answer."}

In [30]:
response=chain.invoke({"query":"How many movies has Tom Hanks acted in?"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {name: 'Tom Hanks'})-[:ACTED_IN]->(m:Movie) RETURN COUNT(m)
Full Context:
[{'COUNT(m)': 2}]

> Finished chain.


{'query': 'How many movies has Tom Hanks acted in?',
 'result': "I don't know the answer."}